In [1]:
import os
import json
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
with open("/workspace/deception-detection/results/repe_lr__repe_honesty__plain__llama-70b-3.3__20251102_210823/control_scores.json", "r") as f:
    control_scores = json.load(f)
    
with open("/workspace/deception-detection/results/repe_lr__repe_honesty__plain__llama-70b-3.3__20251102_210823/scores.json", "r") as f:
    scores = json.load(f)

In [3]:
print(control_scores.keys()), print(scores.keys())

dict_keys(['alpaca__plain'])
dict_keys(['repe_honesty__plain_val', 'roleplaying__plain', 'deception_dilution_repe__plain'])


(None, None)

In [8]:
control_scores_lengths = []
for i in range(len(control_scores["alpaca__plain"]["scores"])):
    control_scores_lengths.append(len(control_scores["alpaca__plain"]["scores"][i]))
    
np.max(control_scores_lengths), np.min(control_scores_lengths)

(3077, 6)

In [4]:
promptwise_control_scores = [np.mean(control_score) for control_score in control_scores['alpaca__plain']["scores"]]
threshold = np.percentile(promptwise_control_scores, 99)
threshold

-5.759691670793349

In [16]:
promptwise_roleplaying_scores = [np.mean(score) for score in scores['roleplaying__plain']["scores"]]
roleplaying_labels = scores['roleplaying__plain']["labels"]
print(len(promptwise_roleplaying_scores), len(roleplaying_labels))

371 371


In [17]:
promptwise_roleplaying_scores_deceptive = []
for score, label in zip(promptwise_roleplaying_scores, roleplaying_labels):
    if label == 'DECEPTIVE':
        promptwise_roleplaying_scores_deceptive.append(score)


In [20]:
all_scores = np.concatenate([promptwise_control_scores, promptwise_roleplaying_scores_deceptive])
all_labels = [0]*len(promptwise_control_scores) + [1]*len(promptwise_roleplaying_scores_deceptive)
gt_labels = [1 if score > threshold else 0 for score in all_scores]

In [21]:
roc_auc_score(all_labels, gt_labels)

0.9752925647013824